In [1]:
#Author: Anshul Kapoor
#Aim: Make Single JSON for all EGamma SFs

#Extract histograms from EGamma standard root files
import uproot4
import math
import numpy as np
sf_Veto = uproot4.open("EGdata/egammaEffi.txt_Ele_Veto_postVFP_EGM2D.root:EGamma_SF2D").to_boost()
sf_Loose = uproot4.open("EGdata/egammaEffi.txt_Ele_Loose_postVFP_EGM2D.root:EGamma_SF2D").to_boost()
sf_Medium = uproot4.open("EGdata/egammaEffi.txt_Ele_Medium_postVFP_EGM2D.root:EGamma_SF2D").to_boost()
sf_Tight = uproot4.open("EGdata/egammaEffi.txt_Ele_Tight_postVFP_EGM2D.root:EGamma_SF2D").to_boost()

In [2]:
#k=uproot4.open("egammaEffi.txt_Ele_Veto_postVFP_EGM2D.root:EGamma_SF2D")

In [3]:
from correctionlib.schemav1 import Correction
sfs=[sf_Veto,sf_Loose,sf_Medium,sf_Tight] # The list of 2D histograms for all wps
names=["Veto","Loose","Medium","Tight"]
nameserr=["Veto_err","Loose_err","Medium_err","Tight_err"]# The labels you want in JSON for all wps
nameJSON="egcorrs_simpler.json" # Name of final JSON
corrs=[None] * len(sfs)

#Use correctionlib to prepare jsons for all wps
for name,sf in zip(names,sfs):
    corr = Correction.parse_obj(
    {
        "version": 0,
        "name": name,
        "inputs": [
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
            {"name": "pt", "type": "real"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": {
            "nodetype": "multibinning",
            "edges": [
                list(sf.axes[0].edges),
                list(sf.axes[1].edges),
            ],
            "content": list(sf.view().value.flatten()),
        },
    })
    exec(name + "= corr")
    
    correrr = Correction.parse_obj(
    {
        "version": 0,
        "name": name + "_err",
        "inputs": [
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
            {"name": "pt", "type": "real"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": {
            "nodetype": "multibinning",
            "edges": [
                list(sf.axes[0].edges),
                list(sf.axes[1].edges),
            ],
            "content": list(np.sqrt(sf.view().variance.flatten())),
        },
    })
    exec(name + "_err = correrr")
    
from correctionlib.schemav1 import CorrectionSet
import gzip

cset = CorrectionSet.parse_obj({
    "schema_version": 1,
    "corrections": [ #Which corrections you want to add to final JSON
        Veto,
        Loose,
        Medium,
        Tight, 
        Veto_err,
        Loose_err,
        Medium_err,
        Tight_err,
    ]
})

with open(nameJSON, "w") as fout:
    fout.write(cset.json(exclude_unset=True, indent=4))

In [4]:
#Evaluator Example

import libcorrection

cset = libcorrection.CorrectionSet(nameJSON)

valsf= cset["Veto"].evaluate(1.1, 34.0)
valsferr= cset["Veto_err"].evaluate(1.1, 34.0)

print("SF is:"+str(valsf))
print("error is:"+str(valsferr))


SF is:0.9864253401756288
error is:1.0
